In [4]:
import torch
from PIL import Image
import requests
from lavis.models import load_model_and_preprocess

In [5]:
from lavis.datasets.builders import load_dataset
msvd_dataset = load_dataset("msvd_qa")

print(msvd_dataset.keys())
print(len(msvd_dataset["train"]))
print(msvd_dataset["train"][0])



Using downloaded and verified file: /home/ommos92/.cache/lavis/msvd/annotations/qa_train.json
Using downloaded and verified file: /home/ommos92/.cache/lavis/msvd/annotations/qa_val.json
Using downloaded and verified file: /home/ommos92/.cache/lavis/msvd/annotations/qa_test.json
Using downloaded and verified file: /home/ommos92/.cache/lavis/msvd/annotations/qa_ans2label.json
dict_keys(['train', 'val', 'test'])
30933
{'video': '/home/ommos92/.cache/lavis/msvd/videos/-4wsuPCjDBc_5_15.avi', 'text_input': 'what is chewing on a nut?', 'answers': 35, 'question_id': '0000000', 'instance_id': '0'}


In [6]:
#Write a function to convert the .avi into a set of images. Then we can randomly 
# sample from this set of images.
import cv2
from PIL import Image
import random

def convert_avi(data):
    # Open the video file
    cap = cv2.VideoCapture(data['video'])

    images = []
    while(cap.isOpened()):
        # Read the video frame by frame
        ret, frame = cap.read()
        if ret:
            # Convert the frame to RGB (OpenCV uses BGR by default)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # Convert the frame to a PIL Image and append to the list
            images.append(Image.fromarray(frame))
        else:
            break

    # Close the video file
    cap.release()

    # Return the list of PIL Images
    return images

def random_sample(images, num_samples):
    #Randomly select 'num_samples' images from the list
    sampled_images = random.sample(images, num_samples)
    return sampled_images


images = convert_avi(msvd_dataset["train"][0])
print(len(images))


300


In [7]:

# setup device to use
device = torch.device("cuda:1") if torch.cuda.is_available() else "cpu"

In [8]:
#Load the model
model, vis_processors, _ = load_model_and_preprocess(
    name="blip2_opt", model_type="pretrain_opt6.7b", is_eval=True, device=device
)


Loading checkpoint shards: 100%|██████████| 2/2 [00:17<00:00,  8.63s/it]


In [ ]:
from lavis.common.vqa_tools.vqa_eval import VQAEval

# Loop over the videos in the dataset
for video in msvd_dataset["test"]:
    # Sample a random frame from the video
    frame = random_sample(convert_avi(video), 1)
    
    # Perform VQA on the frame
    question, predicted_answer = model.predict(frame)
    
    # Get the ground truth from the annotation
    ground_truth = msvd_dataset['annotation'][video]
    ground_truth_question = ground_truth['question']
    ground_truth_answer = ground_truth['answer']
    
    # Ensure the predicted question matches the ground truth question
    assert question == ground_truth_question, "Predicted question does not match ground truth question"
    
    # Create an instance of VQAEval
    vqa_eval = VQAEval(ground_truth_answer, predicted_answer, n=2)
    
    # Compute the accuracy
    vqa_eval.evaluate()
    accuracy = vqa_eval.accuracy["overall"]
    
    # Print or store the accuracy for this video
    print(accuracy)